### RAG on LangChain meets Phoenix


In [35]:
import os 
import openai

import dotenv

from rich import print

dotenv.load_dotenv()


True

In [ ]:
!pip install phoenix

### PromptTemplates

Next stop, we'll discuss a few templates. This allows us to easily interact with our model by not having to redo work we've already completed!

In [36]:
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate
)

# we can signify variables we want access to by wrapping them in {}
system_prompt_template = "You are an expert in {SUBJECT}, and you're currently feeling {MOOD}"
system_prompt_template = SystemMessagePromptTemplate.from_template(system_prompt_template)

user_prompt_template = "{CONTENT}"
user_prompt_template = HumanMessagePromptTemplate.from_template(user_prompt_template)

# put them together into a ChatPromptTemplate
chat_prompt = ChatPromptTemplate.from_messages([system_prompt_template, user_prompt_template])

Now that we have our `chat_prompt` set-up with the templates - let's see how we can easily format them with our content!

NOTE: `disp_markdown` is just a helper function to display the formatted markdown response.

In [37]:
# note the method `to_messages()`, that's what converts our formatted prompt into 
# formatted_chat_prompt = chat_prompt.format_prompt(SUBJECT="cheeses", MOOD="quite tired", CONTENT="Hi, what are the finest cheeses?").to_messages()

# disp_markdown(chat_model(formatted_chat_prompt).content)

### Setting up the LangChain


In [38]:
from langchain.chains import LLMChain
chain = LLMChain(llm=chat_model, prompt=chat_prompt)


### Load up the target book


In [40]:
with open("data/guide1.txt") as f:
    hitchhikersguide = f.read()

Next we'll want to split our text into appropirately sized chunks. 

We're going to be using the [CharacterTextSplitter](https://python.langchain.com/en/latest/modules/indexes/text_splitters/examples/character_text_splitter.html) from LangChain today.

The size of these chunks will depend heavily on a number of factors relating to which LLM you're using, what the max context size is, and more. 

You can also choose to have the chunks overlap to avoid potentially missing any important information between chunks. As we're dealing with a novel - there's not a critical need to include overlap.

We can also pass in the separator - this is what we'll try and separate the documents on. Be careful to understand your documents so you can be sure you use a valid separator!

For now, we'll go with 1000 characters. 

In [41]:
from langchain.text_splitter import CharacterTextSplitter

text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0, separator = "\n")
texts = text_splitter.split_text(hitchhikersguide)

In [42]:
len(texts)

293

Now that we've split our document into more manageable sized chunks. We'll need to embed those documents!

For more information on embedding - please check out [this](https://platform.openai.com/docs/guides/embeddings) resource from OpenAI.

In order to do this, we'll first need to select a method to embed - for this example we'll be using OpenAI's embedding - but you're free to use whatever you'd like. 

You just need to ensure you're using consistent embeddings as they don't play well with others.

In [43]:
from langchain.embeddings.openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings()

Now that we've set up how we want to embed our document - we'll need to embed it. 

For this week we'll be glossing over the technical details of this process - as we'll get more into next week.

Just know that we're converting our text into an easily queryable format!

We're going to leverage ChromaDB for this example, so we'll want to install that dependency. 

In [44]:
# !pip install chromadb tiktoken -q

In [45]:
from langchain.vectorstores import Chroma

docsearch = Chroma.from_texts(texts, embeddings, metadatas=[{"source": str(i)} for i in range(len(texts))]).as_retriever()

Now that we have our documents embedded we're free to query them with natural language! Let's see this in action!

In [46]:
query = "What makes towels important?"
docs = docsearch.get_relevant_documents(query)

In [47]:
docs[0]

Document(page_content="value - you can wrap it around you for warmth as you bound across\nthe cold moons of Jaglan Beta; you can lie on it on the brilliant\nmarble-sanded beaches of Santraginus V, inhaling  the  heady  sea\nvapours;  you can sleep under it beneath the stars which shine so\nredly on the desert world of Kakrafoon; use it  to  sail  a  mini\nraft  down  the slow heavy river Moth; wet it for use in hand-to-\nhand-combat; wrap it round your head to ward off noxious fumes or\nto  avoid  the  gaze of the Ravenous Bugblatter Beast of Traal (a\nmindboggingly stupid animal, it assumes that if you can't see it,\nit  can't  see  you - daft as a bush, but very ravenous); you can\nwave your towel in emergencies  as  a  distress  signal,  and  of\ncourse  dry  yourself  off  with it if it still seems to be clean\nenough.\n \nMore importantly, a towel has immense  psychological  value.  For\nsome reason, if a strag (strag: non-hitch hiker) discovers that a\nhitch hiker has his towel w

Finally, we're able to combine what we've done so far into a chain!

We're going to leverage the `load_qa_chain` to quickly integrate our queryable documents with an LLM.

There are 4 major methods of building this chain, they can be found [here](https://docs.langchain.com/docs/components/chains/index_related_chains)!

For this example we'll be using the `stuff` chain type.

In [48]:
from langchain.chains.question_answering import load_qa_chain
from langchain.llms import OpenAI

chain = load_qa_chain(OpenAI(temperature=0), chain_type="stuff")
query = "What makes towels important?"
chain.run(input_documents=docs, question=query)

' Towels are important because they have immense psychological value and practical uses, such as providing warmth, protection from noxious fumes, and a distress signal.'

Now that we have this set-up, we'll want to package it into an app and pass it to a Hugging Face Space!

You can find instruction on how to do that in the GitHub Repository!